In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import OneHotEncoder

from sklearn.linear_model import LogisticRegression
from sklearn import ensemble
from sklearn.pipeline import make_pipeline
#from sklearn.neural_network import BernoulliRBM

from sklearn.metrics import accuracy_score,log_loss

import sys
sys.path.insert(0,"libs")
import data_cleaning as dc

### Split Given Training Data into Training and Test Sets

In [3]:
train = pd.read_csv("data/train.csv")

train = dc.massage_df(train)

In [4]:
train["age_type"] = train.age_numeric_days.apply(dc.age_type)

In [5]:
# split
mytrain,mytest = train_test_split(train,test_size=0.4)
# furthur split training set
mytrain,mytrain1 = train_test_split(mytrain,test_size=0.4)

### Choose Features and Perform Linear Regression (Logit)

In [7]:
def split_feature_outcome(df,feature_set,outcome_set):
    features = pd.get_dummies(df[feature_set])
    outcomes = df[outcome_set]
    return features,outcomes
# end def split_feature_outcome

In [8]:
features = ["AnimalType","neuter_status","age_type",'is_weekend']
split_it = lambda df:split_feature_outcome(df,features,"OutcomeType")

mytrain_features,mytrain_outcomes   = split_it(mytrain)
mytrain_features1,mytrain_outcomes1 = split_it(mytrain1)
mytest_features,mytest_outcomes     = split_it(mytest)

In [9]:
grd = ensemble.GradientBoostingClassifier(n_estimators=5)
grd_enc = OneHotEncoder()
grd_lm = LogisticRegression()

In [10]:
grd.fit(mytrain_features,mytrain_outcomes);

In [11]:
grd_enc.fit(grd.apply(mytrain_features)[:, :, 0]);

In [12]:
grd_lm.fit(grd_enc.transform(grd.apply(mytrain_features1)[:, :, 0])
           , mytrain_outcomes1);

### Measure Success/Failure

50 percentile on Kaggle has log-loss = 0.84434

In [13]:
prediction_probabilities = grd_lm.predict_proba(
    grd_enc.transform(grd.apply(mytest_features)[:, :, 0])
)
predictions = grd_lm.predict(
    grd_enc.transform(grd.apply(mytest_features)[:, :, 0])
)
# check performance
acc = accuracy_score(mytest_outcomes, predictions)
los = log_loss( pd.get_dummies( mytest.OutcomeType ).values, prediction_probabilities)
print("accuracy = {0:1.2f} \nlog-loss = {1:1.3f}".format(acc,los) )

accuracy = 0.63 
log-loss = 0.977
